In [1]:
from dt_apriltags import Detector
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pymavlink import mavutil
import sys
import signal
from pid import PID


In [2]:

cameraMatrix = np.array([1060.71, 0, 960, 0, 1060.71, 540, 0, 0, 1]).reshape((3, 3))
camera_params = (cameraMatrix[0, 0], cameraMatrix[1, 1], cameraMatrix[0, 2], cameraMatrix[1, 2])
at_detector = Detector(families='tag36h11',
                           nthreads=1,
                           quad_decimate=1.0,
                           quad_sigma=0.0,
                           refine_edges=1,
                           decode_sharpening=0.25,
                           debug=0)


In [3]:

def getTag1(frame, mode = 0):
    # video.set(cv2.CAP_PROP_POS_FRAMES, frameNumber)
    
    
    detected_tags = []
    
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    tags = at_detector.detect(img, True, camera_params, tag_size=0.1)
    for tag in tags:
        (ptA, ptB, ptC, ptD) = tag.corners
        ptB = (int(ptB[0]), int(ptB[1]))
        ptC = (int(ptC[0]), int(ptC[1]))
        ptD = (int(ptD[0]), int(ptD[1]))
        ptA = (int(ptA[0]), int(ptA[1]))
        # draw the bounding box of the AprilTag detection
        cv2.line(frame, ptA, ptB, (0, 255, 0), 3)
        cv2.line(frame, ptB, ptC, (0, 255, 0), 3)
        cv2.line(frame, ptC, ptD, (0, 255, 0), 3)
        cv2.line(frame, ptD, ptA, (0, 255, 0), 3)
        # draw circle in center of tag
        (cX, cY) = (int(tag.center[0]), int(tag.center[1]))
        cv2.circle(frame, (cX, cY), 5, (0, 0, 255), -1)

        # draws line between center and circle
        cv2.line(frame, (cX, cY), (int(frame.shape[1]/2), int(frame.shape[0]/2)), (255, 0, 0), 3)
        detected_tags.append([cX, cY])

        # for idx in range(len(tag.corners)):
        #     cv2.line(frame, tuple(tag.corners[idx - 1, :].astype(int)), tuple(tag.corners[idx, :].astype(int)), (0, 255, 0))
        #     cv2.circle(frame, (x, y), 50, (0, 0, 255), 2)
    if mode == 0: return detected_tags
    if mode == 1: return frame


In [4]:
vida = cv2.VideoCapture('AprilTagTest.mkv')


In [5]:
def tagVideo(vid):
    output_video = cv2.VideoWriter('output_video.avi', cv2.VideoWriter_fourcc(*'XVID'), 30, (1920, 1080))
    # video.release() #Save video to disk.
    # total_frames = []
    # Capture frame-by-frame

    count = 1
    ret, frame = vid.read()

    # change comments for set number of frames
    # while ret:
    while True:
        ret, frame = vid.read()
        if ret:
            output_video.write(getTag1(frame, 1))
        print(ret)
        print(f"Frame: {count}")
        count += 1

        # change comments for set number of frames
        if count > 300:
            break

    output_video.release()


In [6]:

# tagVideo(vida)

In [7]:
# print("[INFO] {} total AprilTags detected".format(len(results)))

# # loop over the AprilTag detection results
# for r in results:
# 	# extract the bounding box (x, y)-coordinates for the AprilTag
# 	# and convert each of the (x, y)-coordinate pairs to integers
# 	(ptA, ptB, ptC, ptD) = r.corners
# 	ptB = (int(ptB[0]), int(ptB[1]))
# 	ptC = (int(ptC[0]), int(ptC[1]))
# 	ptD = (int(ptD[0]), int(ptD[1]))
# 	ptA = (int(ptA[0]), int(ptA[1]))
# 	# draw the bounding box of the AprilTag detection
# 	cv2.line(image, ptA, ptB, (0, 255, 0), 2)
# 	cv2.line(image, ptB, ptC, (0, 255, 0), 2)
# 	cv2.line(image, ptC, ptD, (0, 255, 0), 2)
# 	cv2.line(image, ptD, ptA, (0, 255, 0), 2)
# 	# draw the center (x, y)-coordinates of the AprilTag
# 	(cX, cY) = (int(r.center[0]), int(r.center[1]))
# 	cv2.circle(image, (cX, cY), 5, (0, 0, 255), -1)
# 	# draw the tag family on the image
# 	tagFamily = r.tag_family.decode("utf-8")
# 	cv2.putText(image, tagFamily, (ptA[0], ptA[1] - 15),
# 		cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
# 	print("[INFO] tag family: {}".format(tagFamily))
# # show the output image after AprilTag detection
# cv2.imshow("Image", image)
# cv2.waitKey(0)

In [8]:


def size(video):
    vcap = cv2.VideoCapture(video)
    
    width = int(vcap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(vcap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    middle_x = width/2
    middle_y = height/2

    return middle_x, middle_y



In [9]:


def main():
    video = 'AprilTagTest.mkv'
    #mav = mavutil.mavlink_connection("udpin:0.0.0.0:14550")
    pid = PID(35, 0.0, 10, 2)

    middle_x, middle_y = size(video)  
    x = 0

    try:
        video = cv2.VideoCapture(video)
        while True:
            # Read the current coordinates from the AprilTag detector
            ret, frame = video.read()
            detected_tags = []
            if ret:
                detected_tags = getTag1(frame)
            else:  
                continue
            # if not detected_tags:
            #     print("No tags found in frame", x)
            #     break

            # For simplicity, assume only one tag is detected in each frame
            if len(detected_tags) != 0:
                current_x, current_y = detected_tags[0]
                print(detected_tags)

                # Calculate error from the desired middle coordinates
                error_y = middle_y - current_y
                error_x = middle_x - current_x
        

                print("Frame:", x)
                print("Error Y:", error_y)
                print("Error X:", error_x)
            else:
                print("Frame:", x)
                print("Error Y: No tag detected.")
                print("Error X: No tag detected.")

            # Update the PID controllers and get the output
            # output_y = pid.update(error_y)
            # output_x = pid.update(error_x)

            # print("Output Y:", output_y)
            # print("Output X:", output_x)

            # Set vertical power using the PID output
            #set_vertical_power(mav, -output_y)  # Negative because of the direction of the thruster

            # Set horizontal power using the PID output
            #set_rc_channel_pwm(mav, 6, pwm=1500 + output_x)

            x=x+1
            
    except KeyboardInterrupt:
        print("Interrupted by user.")
        # finally:
        # Stop the vehicle's movement when the program ends
        # set_vertical_power(mav, 0)
        # set_rc_channel_pwm(mav, 6, pwm=1500)



In [10]:

    
if __name__ == "__main__":
    main()

[[1096, 316]]
Frame: 0
Error Y: 224.0
Error X: -136.0
[[1096, 316]]
Frame: 1
Error Y: 224.0
Error X: -136.0
[[1096, 316]]
Frame: 2
Error Y: 224.0
Error X: -136.0
[[1096, 317]]
Frame: 3
Error Y: 223.0
Error X: -136.0
[[1096, 317]]
Frame: 4
Error Y: 223.0
Error X: -136.0
[[1095, 317]]
Frame: 5
Error Y: 223.0
Error X: -135.0
[[1095, 318]]
Frame: 6
Error Y: 222.0
Error X: -135.0
[[1094, 319]]
Frame: 7
Error Y: 221.0
Error X: -134.0
[[1094, 320]]
Frame: 8
Error Y: 220.0
Error X: -134.0
[[1094, 320]]
Frame: 9
Error Y: 220.0
Error X: -134.0
[[1095, 321]]
Frame: 10
Error Y: 219.0
Error X: -135.0
[[1096, 322]]
Frame: 11
Error Y: 218.0
Error X: -136.0
[[1099, 322]]
Frame: 12
Error Y: 218.0
Error X: -139.0
[[1101, 322]]
Frame: 13
Error Y: 218.0
Error X: -141.0
[[1104, 321]]
Frame: 14
Error Y: 219.0
Error X: -144.0
[[1108, 322]]
Frame: 15
Error Y: 218.0
Error X: -148.0
[[1113, 323]]
Frame: 16
Error Y: 217.0
Error X: -153.0
[[1120, 326]]
Frame: 17
Error Y: 214.0
Error X: -160.0
[[1130, 331]]
Frame:

Error, more than one new minimum found.


[[1399, 387]]
Frame: 35
Error Y: 153.0
Error X: -439.0
[[1415, 386]]
Frame: 36
Error Y: 154.0
Error X: -455.0


Error, more than one new minimum found.


[[1429, 385]]
Frame: 37
Error Y: 155.0
Error X: -469.0
[[1440, 384]]
Frame: 38
Error Y: 156.0
Error X: -480.0
[[1448, 385]]
Frame: 39
Error Y: 155.0
Error X: -488.0
[[1456, 385]]
Frame: 40
Error Y: 155.0
Error X: -496.0
[[1464, 385]]
Frame: 41
Error Y: 155.0
Error X: -504.0
[[1468, 385]]
Frame: 42
Error Y: 155.0
Error X: -508.0
[[1471, 385]]
Frame: 43
Error Y: 155.0
Error X: -511.0
[[1473, 384]]
Frame: 44
Error Y: 156.0
Error X: -513.0
[[1472, 384]]
Frame: 45
Error Y: 156.0
Error X: -512.0
[[1470, 384]]
Frame: 46
Error Y: 156.0
Error X: -510.0
[[1467, 384]]
Frame: 47
Error Y: 156.0
Error X: -507.0
[[1462, 383]]
Frame: 48
Error Y: 157.0
Error X: -502.0
[[1456, 383]]
Frame: 49
Error Y: 157.0
Error X: -496.0
[[1450, 382]]
Frame: 50
Error Y: 158.0
Error X: -490.0
[[1446, 381]]
Frame: 51
Error Y: 159.0
Error X: -486.0
[[1446, 380]]
Frame: 52
Error Y: 160.0
Error X: -486.0
[[1447, 380]]
Frame: 53
Error Y: 160.0
Error X: -487.0
[[1452, 380]]
Frame: 54
Error Y: 160.0
Error X: -492.0
[[1459, 38

Error, more than one new minimum found.


[[1225, 420]]
Frame: 93
Error Y: 120.0
Error X: -265.0
[[1210, 421]]
Frame: 94
Error Y: 119.0
Error X: -250.0
[[1195, 422]]
Frame: 95
Error Y: 118.0
Error X: -235.0
[[1180, 422]]
Frame: 96
Error Y: 118.0
Error X: -220.0
[[1167, 423]]
Frame: 97
Error Y: 117.0
Error X: -207.0
[[1152, 423]]
Frame: 98
Error Y: 117.0
Error X: -192.0
[[1138, 423]]
Frame: 99
Error Y: 117.0
Error X: -178.0
[[1124, 423]]
Frame: 100
Error Y: 117.0
Error X: -164.0
[[1110, 423]]
Frame: 101
Error Y: 117.0
Error X: -150.0
[[1096, 422]]
Frame: 102
Error Y: 118.0
Error X: -136.0
[[1082, 421]]
Frame: 103
Error Y: 119.0
Error X: -122.0
[[1068, 421]]
Frame: 104
Error Y: 119.0
Error X: -108.0
[[1055, 420]]
Frame: 105
Error Y: 120.0
Error X: -95.0
[[1041, 419]]
Frame: 106
Error Y: 121.0
Error X: -81.0
[[1028, 418]]
Frame: 107
Error Y: 122.0
Error X: -68.0
[[1015, 416]]
Frame: 108
Error Y: 124.0
Error X: -55.0
[[1002, 414]]
Frame: 109
Error Y: 126.0
Error X: -42.0
[[987, 411]]
Frame: 110
Error Y: 129.0
Error X: -27.0
[[973,

[[836, 383]]
Frame: 120
Error Y: 157.0
Error X: 124.0
[[822, 380]]
Frame: 121
Error Y: 160.0
Error X: 138.0
[[808, 377]]
Frame: 122
Error Y: 163.0
Error X: 152.0
[[795, 374]]
Frame: 123
Error Y: 166.0
Error X: 165.0
[[783, 370]]
Frame: 124
Error Y: 170.0
Error X: 177.0
[[771, 367]]
Frame: 125
Error Y: 173.0
Error X: 189.0
[[759, 364]]
Frame: 126
Error Y: 176.0
Error X: 201.0
[[748, 361]]
Frame: 127
Error Y: 179.0
Error X: 212.0
[[737, 358]]
Frame: 128
Error Y: 182.0
Error X: 223.0
